In [1]:
"""
This is the main code for training and testing. This code is modified from the previous work of our group, names are omitted.
"""


import datetime
import numpy as np
import torch 
import copy
import torchvision
import torch.nn as nn
from torch.autograd import Variable
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.parameter import Parameter
from torch.distributions import Categorical 
import math
import time
import sys
import os
from __future__ import print_function
import json
from datetime import datetime

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib
import matplotlib.backends
from IPython.display import HTML
from matplotlib import rc, colors
import matplotlib.cm as cm
import matplotlib.patches as patches


import argparse
import pickle

#fcns for print and get_time
import fcn as fcn
from skimage.util import view_as_windows

%load_ext autoreload
%autoreload 2

def str2bool(v):
    return v.lower() in ('true', '1')

parser = argparse.ArgumentParser(description="")

# You may choose the number of robots and the operation time in here
parser.add_argument('--n_rollouts', default=1,type=int, help="")
parser.add_argument('--random_seed', default=666662222,type=int, help="")
parser.add_argument('--time_h', default=8,type=int, help="")
parser.add_argument('--f_size', default=4,type=int, help="")
parser.add_argument('--n_agent', default=4,type=int, help="")

parser.add_argument('--log_dir', type=str, default='logsmnist')
parser.add_argument('--gpu', type=int, default=3)
parser.add_argument('--stdout_print', default=True, type=str2bool, help='print control')

args, unknown = parser.parse_known_args()
args = vars(args)

args['log_dir'] = "{}".format(args['log_dir'])
try:
    os.makedirs(args['log_dir'])
except:
    pass

no_cuda = False
use_cuda = not no_cuda and torch.cuda.is_available()

# select your cuda number
device = torch.device("cuda:3" if use_cuda else "cpu")

from torch.utils.data import DataLoader
from torchvision import *

from mnist_penalty_comm import CModel
from mnist_penalty_comm import Actor
from mnist_penalty_comm import EnvMNIST
from mnist_penalty_comm import MnistAgent

import vgg

In [2]:
transform = transforms.Compose([
    transforms.Resize((28,28)),
    transforms.Grayscale(3),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (1.0,))
])


train_set = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)

test_set = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)

bsize = 512

In [3]:
# Please select the apporiate batch_size based on your cuda memory

trainloader = DataLoader(train_set, batch_size=bsize, shuffle=True)
testloader = DataLoader(test_set, batch_size=bsize, shuffle=False)

In [4]:
# Please select the apporiate batch_size based on your cuda memory

batch_size = bsize
f_size=args['f_size']
n_agent=args['n_agent']
n_rollouts=args['n_rollouts'] # 2
prog_print=True
prog_percent=98
time_h=args['time_h']
seed=args['random_seed']
single_train = False
print(args)

{'random_seed': 666662222, 'f_size': 4, 'stdout_print': True, 'log_dir': 'logsmnist', 'gpu': 3, 'n_agent': 4, 'time_h': 8, 'n_rollouts': 1}


In [5]:
vggm = vgg.vgg19(pretrained=True).to(device)
vggm.classifier[6] = nn.Linear(vggm.classifier[6].in_features, 8).to(device)

for param in vggm.features.parameters():
    param.requires_grad = False
print(vggm.classifier[6].out_features)

8


In [6]:
model= CModel(HD=32, frame_size=f_size, n_agent=n_agent, _msg_dim=12, device=device, random_action_flag=False,no_channels=3,edge_pr=1,\
                              kern_size=1,model = vggm).to(device) # HD=64

for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)

model.parameters()

model.classifier.0.weight
model.classifier.0.bias
model.classifier.3.weight
model.classifier.3.bias
model.classifier.6.weight
model.classifier.6.bias
lstm.weight_ih
lstm.weight_hh
lstm.bias_ih
lstm.bias_hh
lstmf1.weight_ih
lstmf1.weight_hh
lstmf1.bias_ih
lstmf1.bias_hh
pred.weight
pred.bias
gn1.weight
gn1.bias
gn2.weight
gn2.bias
gp.weight
gp.bias


<generator object parameters at 0x7fa4775dc2d0>

In [ ]:
for batch_idx, (x, target) in enumerate(trainloader):
#     print(target)
    break
    
for batch_idx, (x, target) in enumerate(trainloader):
#     print(target)
    break
    
def train(eps, optimizer, criterion):
        
    train_cnt = 0    
    best_accuracy = 0
    best_accuracy_train=0
    test_acc_lst = []
    old = datetime.now()
    l_accuracy = 0
    
    for epochs in xrange(eps):
        print(epochs)
        
#         It's recommended to save the model using the following  command
#         torch.save(model.state_dict(), 'mnist_multi.pt')

        for traininput, trainlabel in trainloader:
            x, target = traininput.to(device), trainlabel.to(device)
            optimizer.zero_grad()
            val=-actor.multi_rollout(x.to(device),target.to(device),n_rollouts)
            loss = val 
            loss.backward()
            optimizer.step() #(closure)
            with torch.no_grad():
                # the constraint for training
                max_inf_f = model.inf_f
                # inf norms of the LSTM Wo
                Wo = model.lstm.weight_hh[192:256]
                Wo_inf = torch.norm(Wo,p=1,dim=1)
                inf_Wo = torch.max(Wo_inf)

                if inf_Wo.data.cpu().numpy() >= (1-max_inf_f):
                    model.lstm.weight_hh[192:256] = model.lstm.weight_hh[192:256].clone()/1.2

                # inf norms of the LSTM Wi
                Wi = model.lstm.weight_hh[0:64]
                Wi_inf = torch.norm(Wi,p=1,dim=1)
                inf_Wi = torch.max(Wi_inf)

                if inf_Wi.data.cpu().numpy() >= (1-max_inf_f):
                    
                    model.lstm.weight_hh[0:64] = model.lstm.weight_hh[0:64].clone()/1.2

                # inf norms of the LSTM Wf
                Wf = model.lstm.weight_hh[64:128]
                Wf_inf = torch.norm(Wf,p=1,dim=1)
                inf_Wf = torch.max(Wf_inf)

                if inf_Wf.data.cpu().numpy() >= pow((1-max_inf_f),2):
                    model.lstm.weight_hh[64:128] = model.lstm.weight_hh[64:128].clone()/1.2

                # inf norms of the LSTM Wz
                Wz = model.lstm.weight_hh[128:192]
                Wz_inf = torch.norm(Wz,p=1,dim=1)
                inf_Wz = torch.max(Wz_inf)

                if inf_Wz.data.cpu().numpy() >= 0.25*(1-max_inf_f):
                    model.lstm.weight_hh[128:192] = model.lstm.weight_hh[128:192].clone()/1.2
            

            train_cnt +=1
            if train_cnt%50 == 0:
                correct_cnt = 0
                total_cnt = 0
                with torch.no_grad():
                    for x, target in testloader:
                        actor.multi_rollout(x.to(device),target.to(device),1)
                        preds=actor.final_preds.to(device)
                        _, pred_labels = torch.max(preds,1)
                        correct_cnt += (pred_labels == target.to(device)).sum()
                        total_cnt += pred_labels.shape[0]
                    accuracy = correct_cnt.item() *1.0/(0.0+total_cnt) 
                    l_accuracy = accuracy
                    if accuracy > best_accuracy:
                        best_accuracy = accuracy
                    #print result to file
                    orig_stdout = sys.stdout
                    f = open('mnist_log.txt', 'a+') 
                    sys.stdout = f
                    print ('Epoch', epochs, 'time used', datetime.now()-old,'accuracy',l_accuracy, 'val',val.data.cpu().numpy())
                    f.close()
                    sys.stdout = orig_stdout

    return best_accuracy,test_acc_lst

def test(eps):
    train_cnt = 0    
    best_accuracy = 0
    best_accuracy_train=0
    test_acc_lst = []
    old = datetime.now()
    l_accuracy = 0
    with torch.no_grad():
        for epochs in xrange(eps):
            correct_cnt = 0
            total_cnt = 0
            for x, target in testloader:
                actor.multi_rollout(x.to(device),target.to(device),1)
                preds=actor.final_preds.to(device)
                _, pred_labels = torch.max(preds,1)
                correct_cnt += (pred_labels == target.to(device)).sum()
                total_cnt += pred_labels.shape[0]
            accuracy = correct_cnt.item() *1.0/(0.0+total_cnt) 
            l_accuracy = accuracy
            if accuracy > best_accuracy:
                best_accuracy = accuracy
            print ('Testing accuracy is ',accuracy)
        
    return best_accuracy,test_acc_lst

time_h_list=[time_h] #[1,3,5,7,9,11]

for time_h_counter in range(1):
    time_h=time_h_list[time_h_counter]
    
    for k in range(1): 
        
        model= CModel(HD=32, frame_size=f_size, n_agent=n_agent, _msg_dim=12, device=device, random_action_flag=False,no_channels=3,edge_pr=1,\
                              kern_size=1,model = vggm).to(device) # HD=64
        
        
        env=EnvMNIST(x.to(device),target.to(device),n_agent=n_agent,seed=54,filter_size=f_size,device=device,max_dim=28)
        actor = Actor(model,env,time_horizon=time_h,device=device)
        
        model.lstmcom_flag = False
        
        criterion = nn.CrossEntropyLoss()
        
        optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
        
#         you can load the model using the following command
#         model.load_state_dict(torch.load('mnist_multi.pt',map_location=device))

        actor.random_action_flag = True

#         First training stage, enabled for training with VGG and LSTM
        mx, lst = train(300, optimizer, criterion)

        actor.random_action_flag = False
        
        model.gn1.weight.requires_grad = True
        model.gn1.bias.requires_grad = True
        model.gp.weight.requires_grad = True
        model.gp.bias.requires_grad = True
        
#         Second training stage, enabled for training with goals
        mx, lst = train(200, optimizer, criterion)
        
#         Run this line to get the testing result, this line can be called multiple times 
#         test(1)



mnist_penalty_comm.py:489: UserWarning: torch.range is deprecated in favor of torch.arange and will be removed in 0.5. Note that arange generates values in [start; end), not [start; end].
  x = (torch.range(0, h-1)-(filter_size-1)/2.0)/((filter_size-1)/2.0)
mnist_penalty_comm.py:490: UserWarning: torch.range is deprecated in favor of torch.arange and will be removed in 0.5. Note that arange generates values in [start; end), not [start; end].
  y = (torch.range(0, w-1)-(filter_size-1)/2.0)/((filter_size-1)/2.0)


0


mnist_penalty_comm.py:156: UserWarning: torch.range is deprecated in favor of torch.arange and will be removed in 0.5. Note that arange generates values in [start; end), not [start; end].
  x= torch.stack(mu_goal.shape[0]*[torch.range(0,self.maxDim)],0).to(self.device)
mnist_penalty_comm.py:157: UserWarning: torch.range is deprecated in favor of torch.arange and will be removed in 0.5. Note that arange generates values in [start; end), not [start; end].
  y=torch.stack(mu_goal.shape[0]*[torch.range(0,self.maxDim)],0).to(self.device)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [ ]:
# Robot movement history, square denotes the local obeservation, solid square denotes the goal position, and the green links
# denotes the communication
test(1)
z = model.plot

# change the index_to_play for different testing image
index_to_play=65

print (actor.final_preds[index_to_play],actor.env.targets[index_to_play]) 

actor.env.show_history(index_to_play)